In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
%%time 
df = pd.read_csv('IMDB-Dataset.csv')

CPU times: total: 9.42 s
Wall time: 10.8 s


## Data Cleaning and Preprocessing

In [3]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'so', 'does', 'she', 'o', 'only', 'his', 'during', 'nor', "he'd", 'we', "i've", 'who', 'had', 'how', "you'll", "mightn't", 't', 'll', 'm', 'with', "don't", 'isn', 'no', 'him', 'of', 'wouldn', 'itself', 'and', 'in', "we'd", 'between', 'they', 'below', 'hers', 'than', 'myself', "couldn't", "we've", 'haven', 'ain', "hasn't", "they'll", "aren't", 'its', 'can', 'aren', 'too', "mustn't", "it'll", "she'd", 'or', 've', "wouldn't", 'about', 'but', 'himself', "it'd", "we're", 'being', 'me', 'for', 'our', "she's", 'few', "wasn't", 'doesn', 'more', "didn't", 'were', "you'd", 'further', 'didn', 'doing', "doesn't", 'just', 'once', 'then', 'needn', 'over', "hadn't", 'such', 'these', "we'll", 'am', "needn't", "weren't", 'won', "haven't", 'where', 'any', "they've", 'out', 'themselves', 'there', 'been', "he'll", 'above', 'yourself', 'which', 'again', "they're", 'an', 'was', 'them', 'most', 'down', 'those', 'yourselves', 'to', 'i', 'has', 'a', 'under', 'be', "shan't", 'y', "i'm", 'off', "shouldn't", 'at

In [4]:
def remove_special_character(content):
    return re.sub('\W+',' ', content )
def remove_url(content):
    return re.sub(r'http\S+', '', content)

def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [5]:
%%time
pd.options.display.max_colwidth = 1000
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 46.8 s
Wall time: 49.5 s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* Isenção de responsabilidade: eu só assisti esse filme como um acordo condicional. E eu vejo filmes de graça. Eu não seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crítica mais curta de todos os tempos. Não vê este filme. É de longe o filme mais estúpido, lamenta, preguiçoso e inacreditavelmente UNFUNNY que eu já vi. É um desastre total. Mas como o meu ódio por este filme e por outros, se estende muito além de uma exibição, acho que vou continuar um pouco. Não conheço nenhuma das pessoas do filme além de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso não importa. Eles são todos horríveis, embora eu ache que esse seja o ponto. A edição é horrível e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrível do que eu pensava. Agora eu sei que esses filmes não devem ser sérios, mas vamos lá, é o cinema 101 que se alguém fizer um pequeno corte facial, ele...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

# Feature Engineering

In [6]:
#Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

0    60000
1    60000
Name: Label, dtype: int64


In [7]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

## Lemmatization

In [8]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

## Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

In [9]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with unigram

In [11]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|      able      |   0.105317059622484    |
|   absolutely   |  0.05781602531083612   |
|      act       |  -0.15580785645480488  |
|     acting     |  -0.16873954063674576  |
|     action     |   0.2651399724819559   |
|     actor      |  -0.12856891995919084  |
|    actress     |  -0.1433036551283973   |
|    actually    |  -0.05274917820350205  |
|      add       |  0.027069894150981884  |
|     adult      |  0.056780240473519594  |
|      age       |  0.09895119641091096   |
|     alien      |  -0.07116697616532454  |
|     almost     | -0.015115156560705537  |
|     along      |  0.12030183362158745   |
|    already     |  -0.20680075748588814  |
|      also      |  0.09696559060634195   |
|    although    |   0.195536464012604    |
|     always     |  0.20133047419370823   |
|    amazing     |   0.8406717833923377   |
|    american    |  0.0492444579

## Feature Importance with TFIDF vectorizer and Logistic Regression with Unigram 

In [13]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+-----------------------+
|    Feature     |         Score         |
+----------------+-----------------------+
|      able      |   0.3873871516761897  |
|   absolutely   |   0.498095555318763   |
|      act       |  -1.0560690511284185  |
|     acting     |   -1.622985005197307  |
|     action     |   2.6265345201141153  |
|     actor      |  -1.3172453359008613  |
|    actress     |  -0.9807497508152292  |
|    actually    |  -0.47184070630625835 |
|      add       |  0.14752179114905684  |
|     adult      |   0.7409379285691919  |
|      age       |   0.7765870776472416  |
|     alien      |  -0.3230723881381311  |
|     almost     |  -0.30458144568016904 |
|     along      |   0.7602084807058568  |
|    already     |   -1.159234966912428  |
|      also      |   1.1858604985750933  |
|    although    |   1.221001960049613   |
|     always     |   1.7048274414283464  |
|    amazing     |   4.843524202247544   |
|    american    |   0.4154363964009474  |
|    annoyi

## Vectorization with Count Vectorizer and TDIDF Vectorizer with Bigram

In [14]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Bigram

In [16]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|       acting bad      |  -1.3473047313895938  |
|      acting good      |   0.652059517377615   |
|       acting not      |  -0.45904301506340023 |
|      action film      |  0.37117464012370743  |
|      action movie     |  0.28734603792245483  |
|     action packed     |   1.3417838719146467  |
|      action scene     |  0.17099103965598225  |
|    action sequence    |  0.09735624869629655  |
|     actor actress     |   -0.128830315984452  |
|       actor not       |  -0.3848186677520046  |
|      almost every     |  -0.13568480851713585 |
|       along way       |  0.49011921604419395  |
|        also not       |  -0.13709712517344871 |
|      although not     |   0.5009807086607501  |
|     anything else     |  -0.31600350316174347 |
|        b movie        |  -0.09113046136742965 |
|       bad acting      |  -1.0494655708840284  |


## Feature Importance with Logistic Regression and TFIDF Vectorizer with Bigram

In [17]:
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=50:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+----------------------+
|        Feature        |        Score         |
+-----------------------+----------------------+
|       acting bad      |  -2.93940404939261   |
|      acting good      |  1.4547603119265098  |
|       acting not      | -1.2738035388219235  |
|      action film      |  1.219661641005472   |
|      action movie     |  0.8613687650433228  |
|     action packed     |  2.9478807708416443  |
|      action scene     | 0.47348816246627157  |
|    action sequence    | 0.29274891755657423  |
|     actor actress     | -0.26274460758639234 |
|       actor not       | -0.9351998447191149  |
|      almost every     | -0.3017362917735462  |
|       along way       |  1.1003041482019142  |
|        also not       | -0.2978873089412741  |
|      although not     |  1.1224408463112925  |
|     anything else     | -0.8002445454687251  |
|        b movie        | -0.43025002898906867 |
|       bad acting      | -2.7782826104777225  |
|        bad bad    

In [18]:
pd.options.display.max_colwidth = 1000
df[["Reviews","Ratings","Movies"]][(df['Ratings']>=9)&(df['Reviews_clean'].str.contains("bad review"))].head(100)

,Reviews,Ratings,Movies
120047,"While I wouldn't call this the greatest movie ever made, it's not anywhere near as bad as other reviewers have made it out to be. An average rating of 5 or 6 stars would be fair, but 1.5 is harsh and totally undeserved.Ring of Terror feels like an episode of The Twilight Zone stretched to an hour. In fact, it's so much like a TV show that one wonders if it might not have been originally created as a pilot.If you're a fan of 1950s horror/suspense series like Thriller, The Veil, One Step Beyond, Tales of Tomorrow, and Alfred Hitchcock Presents, you'll likely find this a pleasant way to spend an hour, as I did.Normally I would only give this film 6 out of 10 stars, but because others have been panning it so unmercifully, I'm giving it a 9.",9.0,Ring of Terror
120211,"This movie was a blast for my little guys, they loved every minute of it, I have read all of the bad reviews, and could not disagree more. This movie, is pure and good. There is just enough action to keep the kids interested, and not so much that you leave the theater with them bouncing off the walls either. It is funny with jokes that everyone can appreciate. I think people have gotten used to so much violence and adult content in our kids movies that they are disappointed when it is missing, like the movie wasn't entertaining enough for the parents. Well, NEWS FLASH.....It's a kids movie, and a perfect one at that. Kids need these kinds of movies, not Spongebob and the like which are more to entertain the parents.",9.0,Doogal
120238,"I am a huge horror buff and prefer pieces that delve into the characters psychological issues. This film was awesome on so many levels, the acting and writing were fantastic and creepy and I was afraid or and empathetic with the murderer the whole time. What an interesting study on the line between sick and a danger to others, and the line between being a mean girl and being psychotic. Set in a great location, a house full of creepy art, in the winter in Conneticut and with amazing performances from many of my favorite actresses. It actually shocks me that others have given this such a bad review, I loved this movie, I guess it goes to show you everyone will have a different opinion but I say don't miss this film!",9.0,#Horror
120239,"No idea why there are so many bad reviews here? I loved it; I thought it was a very advanced thoughtful film. The graphic were #killer. The comparison of video game culture and young girl culture was spot on. This film makes connections that I've never seen on the big screen but, do see in every day life.The casting was spot on, Hello 12 year-old girls are supposed to be a little annoying. I do wish that more directors would take color into more consideration the way this film does. T The highly stylized sets make the murder scenes more believable because everything is so unbelievable. How can you live in 2016 and not ""get""a film about social media and accelerationism. #duh Someone explain this to me.",9.0,#Horror
120273,"What do you get when you cross Love Story with Star Wars with Blade Runner with Back to the Future with MTV? Love Story 2050, that's what. What a fun movie for the entire family. This fantasy of epic proportions is much, much better than AI, a similar sci-fi classic. The thrills are non-stop in this blockbuster, from its lead off car chase to bike racing stunts to the vantage point of a moving roller coaster to speeding hover-craft--you will be on the edge of your seat from beginning to end. The version I saw was only partially in English and I still was glued to the screen. I can't wait to see a version with subtitles. The mega budget special effects are out of this world and highly convincing. The future vision of XBox was hilarious. Those who are complaining about how long this movie is simply don't understand Bollywood. The three hours went by quickly; it seemed to be only an hour. There could have been a better twist with the Darth Vader character. 

## Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram

In [19]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Trigram

In [20]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+-----------------------+
|          Feature          |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |  -0.4093565358949509  |
|      acting not good      |  -1.0947661699604188  |
|      acting not great     |  -0.48963402867523753 |
|     acting pretty good    |   0.9940737591058084  |
|      acting top notch     |   1.814143716752449   |
|      action movie not     |   1.0816581259130575  |
|      action set piece     |   1.2481772593191869  |
|     action take place     |  0.03729759325905406  |
|       actor good job      |   0.6164565865776275  |
|    actually pretty good   |  0.30748392905357586  |
|    actually quite good    |  -0.15925005476424173 |
|     almost every scene    |  -0.3251556560378008  |
|        b movie not        | -0.045597121320339266 |
|       bad acting bad      |  -2.3524388933027294  |
|        bad bad bad        |   -1.801864837444694  |
|        bad guy not        

## Feature Importance with Logistic Regression and TFIDF Vectorizer with Trigram

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+------------------------+
|          Feature          |         Score          |
+---------------------------+------------------------+
|       acting not bad      |  -0.5644536233074348   |
|      acting not good      |  -1.3622211891724412   |
|      acting not great     |  -0.6796635736797167   |
|     acting pretty good    |   1.1318093710649246   |
|      acting top notch     |   2.0593395078970413   |
|      action movie not     |   1.277425285075967    |
|      action set piece     |   1.2631913554023428   |
|     action take place     |  0.08811694277626066   |
|       actor good job      |   0.6919829791057166   |
|    actually pretty good   |   0.3414700366790171   |
|    actually quite good    |  -0.13838040873959154  |
|     almost every scene    |  -0.38420230589468085  |
|        b movie not        |  -0.09679572629844871  |
|       bad acting bad      |   -2.934333439657907   |
|        bad bad bad        |  -2.4784010586366074   |
|        b

## Vectorization with Count Vectorizer and TDIDF Vectorizer with 4-gram

In [22]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with 4-gram

In [23]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |  -2.0305373797412054  |
|           acting also great special            |  0.49592889838526555  |
|             acting bad script bad              |  -1.0134287864350715  |
|     addictive consistently compelling show     |   0.2800025822524592  |
|                 ai not gon na                  |  -0.4958879456636554  |
|             also great acting also             |   0.4066244590548684  |
|           also great special effect            |   0.8377584418498645  |
|       annoying illogical decision making       |  -0.2949137045442379  |
|           another film seen recently           |  -0.3230966380336749  |
|         appreciation horror low budget         |  -0.28261741199627427 |
|        atmosphere drear

## Feature Importance with Logistic Regression and TDIDF Vectorizer with 4-gram

In [24]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |   -2.182400596183893  |
|           acting also great special            |   0.6326189210515983  |
|             acting bad script bad              |  -1.2342003392050618  |
|     addictive consistently compelling show     |  0.23278467724397311  |
|                 ai not gon na                  |  -0.4785226891684258  |
|             also great acting also             |   0.5357901976012579  |
|           also great special effect            |   0.9167520189146885  |
|       annoying illogical decision making       |  -0.22430050315743974 |
|           another film seen recently           |  -0.2438149106873676  |
|         appreciation horror low budget         |  -0.2571733277212104  |
|        atmosphere drear

## Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram

In [25]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=5000)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Selection with Chi squared

In [26]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'